<hr>
<font size="6"> <b> Uploading to Image Services </b></font>

<font size="5"> Upload Archived Maps in S3 Bucket to Image Service </font>

<br>
<font size="4"> <b> HydroSAR Transition Workshop:</b> HydroSAR Data Products and Web Application Training <br>
<font size="4"> <b> PI:</b> Franz J. Meyer <br>
<br>
<br>


In [1]:
#Import relevant Python packages
import argparse
import datetime
import json
import logging
import os
import subprocess
import tempfile

import arcpy

ModuleNotFoundError: No module named 'arcpy'

Select configuration file

In [ ]:
config_file = '/Users/jrsmale/GitHub/hyp3-nasa-disasters/data_management/hkh_watermaps.json'

Set up cutoff dates.

In [ ]:
cutoff_date = datetime.datetime.now(tz=datetime.timezone.utc) - datetime.timedelta(days=config['time_period_in_days'])
crf_name = 'Ovi_' + str(datetime.datetime.now(tz=datetime.timezone.utc).strftime('%Y%m%dT%H%M%S')) + '.crf'

print(f'Cutoff date is {cutoff_date}, \n and the crf name is named {crf_name}')

Update source mosaic

In [ ]:
source_dataset = config['datasets']

arcpy.management.RemoveRastersFromMosaicDataset(in_mosaic_dataset=dataset['source_dataset'],
                                                where_clause='OBJECTID>=0'
                                                )
arcpy.management.AddRastersToMosaicDataset(in_mosaic_dataset=dataset['source_dataset'],
                                           raster_type='Raster Dataset',
                                           input_path=dataset['source_dataset'],
                                           update_cellsize_ranges='NO_CELL_SIZES',
                                           filter=dataset['raster_filter']
                                           )
arcpy.management.CalculateFields(
            in_table=dataset['source_mosaic'],
            fields=[
                ['StartDate', "datetime.datetime.strptime(!Name!.split('_')[2], '%Y%m%dT%H%M%S')"],
                ['EndDate', "datetime.datetime.strptime(!Name!.split('_')[2], '%Y%m%dT%H%M%S')"],
            ],
        )
date_sel = f"StartDate <= timestamp '{cutoff_date.strftime('%Y-%m-%d %H:%M:%S')}'"
arcpy.management.RemoveRastersFromMosaicDataset(in_mosaic_dataset=dataset['source_mosaic'],
                                                        where_clause=date_sel)
arcpy.management.CalculateFields(
            in_table=dataset['source_mosaic'],
            fields=[
                ['GroupName', '"_".join(!Name!.split(";")[0].split("_")[:-1])'],
                ['Tag', '!Name!.split("_")[8]'],
                ['MinPS', '0'],
                ['MaxPS', '1610'],
            ],
        )

Create overview crf file with source mosaic

In [ ]:
s3_crf_key = os.path.join(dataset['overview_location'], crf_name)
with tempfile.TemporaryDirectory(dir=config['raster_store']) as temp_dir:
    local_crf = os.path.join(temp_dir, crf_name)
    with arcpy.EnvManager(pyramid='PYRAMIDS 3', cellSize=900):
        arcpy.management.CopyRaster(in_raster=dataset['source_mosaic'], out_rasterdataset=local_crf)
    subprocess.run(['aws', 's3', 'cp', local_crf, s3_crf_key.replace('/vsis3/', 's3://'), '--recursive'])

Adding overview file to source mosaic

In [ ]:
arcpy.management.AddRastersToMosaicDataset(
            in_mosaic_dataset=dataset['source_mosaic'],
            raster_type='Raster Dataset',
            input_path=s3_crf_key,
            update_cellsize_ranges='NO_CELL_SIZES',
        )
selection = arcpy.management.SelectLayerByAttribute(
            in_layer_or_view=dataset['source_mosaic'],
            selection_type='NEW_SELECTION',
            where_clause="Name LIKE 'cr%'",
        )
arcpy.management.CalculateFields(
            in_table=selection,
            fields=[
                ['StartDate', cutoff_date.strftime("'%m/%d/%Y %H:%M:%S'")],
                ['EndDate', 'datetime.datetime.now(tz=datetime.timezone.utc)'],
                ['MinPS', '1600'],
                ['MaxPS', '18000'],
                ['Category', '2'],
                ['GroupName', "'Mosaic Overview'"],
            ],
        )

Updating derived mosaic

In [ ]:
arcpy.management.RemoveRastersFromMosaicDataset(in_mosaic_dataset=dataset['derived_mosaic'],
                                                where_clause='OBJECTID>=0')
arcpy.management.AddRastersToMosaicDataset(
    in_mosaic_dataset=dataset['derived_mosaic'],
    raster_type='Table / Raster Catalog',
    input_path=dataset['source_mosaic'],
    update_cellsize_ranges='NO_CELL_SIZES',
)
selection = arcpy.management.SelectLayerByAttribute(
    in_layer_or_view=dataset['derived_mosaic'],
    selection_type='NEW_SELECTION',
    where_clause="Name NOT LIKE 'Ovi_%'",
)
arcpy.management.CalculateFields(
    in_table=selection,
    fields=[
        ['MinPS', '0'],
    ],
)

Building boundary file for the referenced mosaic

In [ ]:
arcpy.management.BuildBoundary(in_mosaic_dataset=dataset['referenced_mosaic'])

Finished